In [31]:
# Import library
import re
import os
import shutil
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr

In [21]:
# Ambil file asli dari folder
directory_path_real = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap7(DataCleaning)'

files_real = os.listdir(directory_path_real)
list_files_real = []

# Ambil daftar nama file
for file in files_real:
    if '.DS_Store' not in file:
        list_files_real.append(file)
        
all_text_real = ""
for file in list_files_real:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap7(DataCleaning)/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_real += content


In [22]:
# Ambil file parafrasa dari folder
directory_path_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap9(ParaphrasedPaper)'

files_paraphrased = os.listdir(directory_path_paraphrased)
list_files_paraphrased = []

# Ambil daftar nama file
for file in files_paraphrased:
    if '.DS_Store' not in file:
        list_files_paraphrased.append(file)
        
all_text_paraphrased = ""
for file in list_files_paraphrased:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap9(ParaphrasedPaper)/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_paraphrased += content
        


In [23]:
# Melakukan lowercase lagi
all_text_real = all_text_real.lower()
all_text_paraphrased = all_text_paraphrased.lower()

In [24]:
# Menggabungkan teks asli dan parafrasa
all_text = all_text_real + '\n' + all_text_paraphrased

In [25]:
# Hanya mengambil huruf dan titik
all_text = re.sub(r'[^a-zA-Z.]', ' ', all_text)
all_text = re.sub(r'\s+', ' ', all_text)

In [27]:
# Buat file sementara
with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', 'w') as file:
    file.write(all_text)

In [28]:
# Membuat model word embedding fasttext menggunakan CBOW
model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/dataCurr.txt', model='cbow', lr=0.01, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)


Read 5M words
Number of words:  77333
Number of labels: 0
Progress: 100.0% words/sec/thread:  131981 lr:  0.000000 avg.loss:  1.639794 ETA:   0h 0m 0s  0.3% words/sec/thread:  215333 lr:  0.009972 avg.loss:  3.069705 ETA:   0h 4m20s  0.5% words/sec/thread:  234088 lr:  0.009949 avg.loss:  2.746417 ETA:   0h 3m59s  4.6% words/sec/thread:  215173 lr:  0.009535 avg.loss:  2.222255 ETA:   0h 4m 9s 10.3% words/sec/thread:  203078 lr:  0.008965 avg.loss:  2.058223 ETA:   0h 4m 8s 11.2% words/sec/thread:  196004 lr:  0.008877 avg.loss:  2.049758 ETA:   0h 4m15s words/sec/thread:  175901 lr:  0.008657 avg.loss:  2.030978 ETA:   0h 4m37s 13.8% words/sec/thread:  173840 lr:  0.008623 avg.loss:  2.028647 ETA:   0h 4m39s 14.4% words/sec/thread:  170374 lr:  0.008557 avg.loss:  2.024665 ETA:   0h 4m43s lr:  0.008479 avg.loss:  2.020470 ETA:   0h 4m48s 15.9% words/sec/thread:  162876 lr:  0.008414 avg.loss:  2.016406 ETA:   0h 4m51s 17.0% words/sec/thread:  159214 lr:  0.008296 avg.loss:  2.008107 E

In [37]:
# Cek jumlah kata dari data evaluasi yang ada dan tidak ada pada model word embedding word2vec cbow 1

dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_words_exist = []
list_words_not_exist = []


with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        
        # Cek kata pertama
        try:
            word_vector = model_cbow.get_word_vector(word_1)
            if word_1 not in list_words_exist:
                list_words_exist.append(word_1)
        except:
            if word_1 not in list_words_not_exist:
                list_words_not_exist.append(word_1)
        
        # Cek kata kedua
        try:
            word_vector = model_cbow.get_word_vector(word_2)
            if word_2 not in list_words_exist:
                list_words_exist.append(word_2)
        except:
            if word_2 not in list_words_not_exist:
                list_words_not_exist.append(word_2)

print("persentase kata ada", len(list_words_exist) / (len(list_words_exist) + len(list_words_not_exist)) )
print("persentase kata tidak ada", len(list_words_not_exist) / (len(list_words_exist) + len(list_words_not_exist)))

print(list_words_exist)
print(list_words_not_exist)

persentase kata ada 1.0
persentase kata tidak ada 0.0
['computer', 'keyboard', 'jerusalem', 'israel', 'planet', 'galaxy', 'canyon', 'landscape', 'opec', 'country', 'day', 'summer', 'dawn', 'citizen', 'people', 'environment', 'ecology', 'maradona', 'football', 'oil', 'money', 'bank', 'software', 'law', 'lawyer', 'weather', 'forecast', 'network', 'hardware', 'nature', 'fbi', 'investigation', 'wealth', 'psychology', 'freud', 'news', 'report', 'war', 'troops', 'physics', 'proton', 'stock', 'market', 'constellation', 'credit', 'card', 'hotel', 'reservation', 'closet', 'clothes', 'soap', 'opera', 'astronomer', 'space', 'movie', 'theater', 'treatment', 'recovery', 'baby', 'mother', 'deposit', 'television', 'film', 'mind', 'game', 'team', 'admission', 'ticket', 'palestinian', 'arafat', 'terror', 'boxing', 'round', 'internet', 'property', 'tennis', 'racket', 'telephone', 'communication', 'currency', 'cognition', 'seafood', 'sea', 'book', 'paper', 'library', 'depression', 'fighting', 'defeating'

In [47]:
# Baca file wordsim_relatedness_goldstandard untuk evaluasi word embedding dengan teknik relatedness cbow
dir_path_wordsim_relatedness_goldstandard = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataForEvaluatinWordEmbedding/Relatedness/wordsim_relatedness_goldstandard.txt'
list_model_similarity = []

list_human_score = []
with open(dir_path_wordsim_relatedness_goldstandard , 'r') as file:
    content = file.read()
    lines = content.splitlines()
    for line in lines:
        item_in_line = line.split("\t")
        word_1 = item_in_line[0].lower()
        word_2 = item_in_line[1].lower()
        human_score = float(item_in_line[2])
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model_cbow.get_word_vector(word_1))
            wv_2.append(model_cbow.get_word_vector(word_2))
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            list_human_score.append(human_score)
        except:
            continue
            #print("word_1", word_1)
            #print("word_2", word_2)

rho_model, p_model = spearmanr(list_model_similarity, list_human_score)


In [48]:
print(rho_model)

0.12742615864701087
